In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Import your Gradient Descent classes
from rice_ml.optimization.gradient_descent import (
    GradientDescent1D, GradientDescentND
)

In [ ]:
# Create a synthetic regression dataset
X, y = make_regression(
    n_samples=300,
    n_features=2,
    noise=15,
    random_state=42
)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

X_train[:5], y_train[:5]

In [ ]:
def mse_loss(X, y, w):
    predictions = X @ w
    return np.mean((predictions - y)**2)

def mse_gradient(X, y, w):
    predictions = X @ w
    grad = (2 / len(y)) * X.T @ (predictions - y)
    return grad

In [ ]:
# Initialize weights
w0 = np.zeros(X_train.shape[1])

# Create GD optimizer
gd = GradientDescentND(
    df=lambda w: mse_gradient(X_train, y_train, w),
    alpha=0.1,
    tol=1e-6,
    max_iter=2000
)

path, w_final = gd.fit(w0)
w_final

In [ ]:
losses = [mse_loss(X_train, y_train, w) for w in path]

plt.figure(figsize=(7,5))
plt.plot(losses, linewidth=2)
plt.title("Gradient Descent Learning Curve (MSE)")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

In [ ]:
# Closed-form solution: w = (X^T X)^(-1) X^T y
w_closed = np.linalg.inv(X_train.T @ X_train) @ (X_train.T @ y_train)

print("Gradient Descent Weights:", w_final)
print("Closed Form Weights:", w_closed)

In [ ]:
def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - ss_res/ss_tot

y_pred_gd = X_test @ w_final
y_pred_closed = X_test @ w_closed

print("R2 (Gradient Descent):", r2_score(y_test, y_pred_gd))
print("R2 (Closed Form):    ", r2_score(y_test, y_pred_closed))

In [ ]:
# Make grid for contour plot
w1 = np.linspace(w_final[0] - 2, w_final[0] + 2, 200)
w2 = np.linspace(w_final[1] - 2, w_final[1] + 2, 200)

W1, W2 = np.meshgrid(w1, w2)
Loss = np.zeros_like(W1)

for i in range(W1.shape[0]):
    for j in range(W1.shape[1]):
        w_temp = np.array([W1[i, j], W2[i, j]])
        Loss[i, j] = mse_loss(X_train, y_train, w_temp)

# Convert GD path to arrays
path_arr = np.array(path)

plt.figure(figsize=(8,6))
plt.contourf(W1, W2, Loss, levels=50, cmap="viridis")
plt.colorbar(label="Loss (MSE)")
plt.plot(path_arr[:, 0], path_arr[:, 1], color="white", linewidth=2)
plt.scatter(path_arr[0,0], path_arr[0,1], color="red", label="Start")
plt.scatter(path_arr[-1,0], path_arr[-1,1], color="yellow", label="End")
plt.legend()
plt.title("Gradient Descent Path on Loss Surface")
plt.xlabel("Weight 1")
plt.ylabel("Weight 2")
plt.show()